In [9]:
import datacube
import odc.algo
import numpy as np
import matplotlib.pyplot as plt

In [3]:
dc = datacube.Datacube(app = "BAIS2")

In [4]:
aoi = {
    'longitude': (80.78694696, 81.02203692),
    'latitude': (15.75418332, 15.85828652),
    'time': ('2022-01-01', '2023-12-31')
}
ds = dc.load(product='s2a_sen2cor_granule',
             measurements=['scl', 'red', 'green', 'blue', 'B06_20m', 'B07_20m', 'B12_20m', 'B8A_20m'],
             resolution = (-30, 30),
             output_crs = "EPSG:3577",
             **aoi)
print(ds)

<xarray.Dataset>
Dimensions:      (time: 2, y: 691, x: 1080)
Coordinates:
  * time         (time) datetime64[ns] 2022-08-23T05:14:41.977841 2023-02-09T...
  * y            (y) float64 1.859e+05 1.859e+05 ... 1.653e+05 1.652e+05
  * x            (x) float64 -6.623e+06 -6.622e+06 ... -6.59e+06 -6.59e+06
    spatial_ref  int32 3577
Data variables:
    scl          (time, y, x) uint8 10 10 10 10 10 10 10 10 ... 6 6 6 6 6 6 6 6
    red          (time, y, x) uint16 2220 2285 2348 2323 ... 1144 1160 1149 1147
    green        (time, y, x) uint16 2315 2532 2302 2392 ... 1282 1287 1277 1277
    blue         (time, y, x) uint16 2047 1900 2097 2040 ... 1344 1320 1322 1311
    B06_20m      (time, y, x) uint16 2670 2792 2703 2532 ... 1116 1120 1124 1136
    B07_20m      (time, y, x) uint16 2723 2836 2801 2552 ... 1115 1140 1112 1136
    B12_20m      (time, y, x) uint16 1619 1646 1671 1690 ... 1059 1052 1059 1057
    B8A_20m      (time, y, x) uint16 2674 2750 2679 2492 ... 1108 1112 1118 1128
Attrib

In [5]:
# convert dataset to float32 datatype so no-data values are set to NaN
dataset = ds
dataset =  odc.algo.to_f32(dataset)

# Calculate the components that make up the NDVI calculation
# (1-((B06*B07*B8A)/B04)**0.5)*((B12-B8A)/((B12+B8A)**0.5)+1)
a = (1-((dataset.B06_20m*dataset.B07_20m*dataset.B8A_20m)/dataset.red)**0.5)
b = ((dataset.B12_20m-dataset.B8A_20m)/((dataset.B12_20m+dataset.B8A_20m)**0.5)+1)

# Calculate NDVI and store it as a measurement in the original dataset
ndvi = a*b
print(ndvi)

<xarray.DataArray (time: 2, y: 691, x: 1080)>
array([[[ 4.46748438e+04,  4.82986914e+04,  4.19658789e+04, ...,
          2.75172534e+03,  2.12426001e+03,  1.69931995e+03],
        [ 4.22911953e+04,  4.59753203e+04,  4.62381602e+04, ...,
          2.93703687e+03,  2.89363550e+03,  3.22895117e+03],
        [ 4.45524570e+04,  4.62960508e+04,  6.31610000e+04, ...,
          2.08383887e+03,  3.03008496e+03,  3.24639062e+03],
        ...,
        [ 2.25883057e+03,  2.68310010e+03,  2.11054370e+03, ...,
          5.14987488e+02, -2.12968567e+02,  4.97169952e+02],
        [ 1.80355493e+03,  3.27459717e+03,  1.91205261e+03, ...,
          5.85500793e+02,  7.17775513e+02,  1.18131493e+02],
        [ 2.15712622e+03,  2.42963306e+03,  2.19342725e+03, ...,
          5.30428589e+02,  1.07132141e+03,  1.48367285e+03]],

       [[ 4.77643938e+05,  2.35182047e+05,  1.54944891e+05, ...,
          4.18523102e+02, -5.41081909e+02, -2.89718048e+02],
        [ 3.36031812e+05,  2.41970328e+05,  1.27252469e+0

In [11]:
min = 0;
max = 0.99;
zero = 0.5;

# // colorBlend will return a color when the index is between min and max and white when it is less than min.
# // To see black when it is more than max, uncomment the last line of colorBlend.
# // The min/max values were computed automatically and may be poorly specified, feel free to change them to tweak the displayed range.
# // This index crosses zero, so a diverging color map is used. To tweak the value of the break in the color map, change the variable 'zero'.

underflow_color = [1, 1, 1]
low_color = [0/255, 0/255, 255/255]
high_color = [255/255, 20/255, 20/255]
zero_color = [250/255, 255/255, 10/255]
overflow_color = [255/255, 0/255, 255/255]
index = (1-((dataset.B06_20m*dataset.B07_20m*dataset.B8A_20m)/dataset.red)**0.5)*((dataset.B12_20m-dataset.B8A_20m)/((dataset.B12_20m+dataset.B8A_20m)**0.5)+1)
def colorBlend(index, positions, colors):
    """
    Returns a color when the index is between positions and white when it is less than positions.
    To see black when it is more than the maximum position, uncomment the last line of this function.
    """
    if index < positions[0]:
        return colors[0]
    elif index >= positions[-1]:
        # Uncomment to see black when it is more than the maximum position
        # return [0, 0, 0]
        return colors[-1]
    else:
        for i in range(len(positions) - 1):
            if index >= positions[i] and index < positions[i+1]:
                return colors[i] + (colors[i+1] - colors[i]) * ((index - positions[i]) / (positions[i+1] - positions[i]))[:3]

# index = (1-((B06*B07*B8A)/B04)**0.5)*((B12-B8A)/((B12+B8A)**0.5)+1)

print(colorBlend(np.array(index).all(), [min, min, zero, max],
[
underflow_color,
low_color,
zero_color, # divergent step at zero
high_color,
overflow_color # uncomment to see overflows
]))

plt.figure(figsize=(8, 8))
ndvi.plot(col='time', vmin=0, vmax=1, col_wrap=3, cmap = "plasma")
plt.show()

TypeError: unsupported operand type(s) for -: 'list' and 'list'